In [ ]:
# Imports
import sys
sys.path.append("Program")

import datetime as dt
from fundamentals import *
from helper_functions import get_current_date, generate_end_dates, merge_stocks
import pandas as pd
pd.options.mode.chained_assignment = None
from plot import *
from stock_screener import create_rs_volume_df, stoploss_target
from technicals import *

In [ ]:
# Start of the program
start = dt.datetime.now()

# Initial setup
current_date = get_current_date(start)

# Variables
NASDAQ_all = False
period_hk = 60 # Period for HK stocks
period_us = 252 # Period for US stocks
RS = 90
factors = [1, 1, 1]
backtest = False

# Index
index_name = "^HSI"
index_dict = {"^HSI": "HKEX", "^GSPC": "S&P 500", "^IXIC": "NASDAQ Composite"}

In [ ]:
# Get the stock list of HSI
df = pd.read_csv("Program/constituents-hsi.csv")
stocks = df["Symbol"].tolist()

In [ ]:
# Get the infix
infix = get_infix(index_name, index_dict, NASDAQ_all)

# Define the result folder
result_folder = "Result"

 # Get the price data of the index
index_df = get_df(index_name, current_date)

In [ ]:
# Combine the long term and short term RS dataframes
def long_short_RS(end_date1, end_date2, period1, period2):
    # Filter the data
    index_df1 = index_df[index_df.index <= end_date1]
    index_df2 = index_df[index_df.index <= end_date2]

    # Calculate the percent change of the index
    index_df1.loc[:, "Percent Change"] = index_df1["Close"].pct_change()
    index_df2.loc[:, "Percent Change"] = index_df2["Close"].pct_change()
    
    # Calculate the total return of the index
    index_return1 = (index_df1["Percent Change"] + 1).tail(period1).cumprod().iloc[-1]
    index_return2 = (index_df2["Percent Change"] + 1).tail(period2).cumprod().iloc[-1]
    index_shortName = index_dict[f"{index_name}"]
    print(f"Return for {index_shortName} between {index_df1.index[-period1].strftime('%Y-%m-%d')} and {end_date1}: {index_return1:.2f}")
    print(f"Return for {index_shortName} between {index_df2.index[-period2].strftime('%Y-%m-%d')} and {end_date2}: {index_return2:.2f}")

    rs_df1, _, _ = create_rs_volume_df(stocks, end_date1, current_date, period1, index_return1, index_shortName, result_folder, infix, True)
    rs_df2, _, _ = create_rs_volume_df(stocks, end_date2, current_date, period2, index_return2, index_shortName, result_folder, infix, True)

    # Merge the two dataframes on "Ticker"
    merged_df = pd.merge(rs_df1, rs_df2, on="Ticker", suffixes=(" 1", " 2"))

    # Rename the columns
    merged_df = merged_df.rename(columns={"RS 1": "Long-term RS", "RS 2": "Short-term RS"})

    # Drop NaN values
    merged_df = merged_df.dropna()

    return merged_df

In [ ]:
# Assuming merged_df is your DataFrame
merged_df = long_short_RS("2024-09-12", "2024-10-07", 252, 20)

# Scatter plot of short-term RS against long-term RS
plt.figure(figsize=(10, 6))
plt.scatter(merged_df["Long-term RS"], merged_df["Short-term RS"], color="blue", marker="x")

# Highlight the specific ticker with a star
star = merged_df[merged_df["Ticker"] == "9618.HK"]
if not star.empty:
    plt.scatter(star["Long-term RS"], star["Short-term RS"], color="gold", edgecolor="black", marker="*", s=100, label="9618.HK")

# Set the axes labels
plt.xlabel("Long-term RS")
plt.ylabel("Short-term RS")

# Set the title
plt.title(f"Long-term vs Short-term RS (2024-09-12 to 2024-10-07)")

# Set the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Define the end dates
end_dates = generate_end_dates(1, current_date)

In [ ]:
# Iterate over all end dates
for i in range(len(end_dates) - 1):
    start_date = end_dates[i]
    end_date = end_dates[i + 1]

    # Get the merged dataframe
    merged_df = long_short_RS(start_date, end_date, 252, 20)

    # Scatter plot of short-term RS against long-term RS
    plt.figure(figsize=(10, 6))
    plt.scatter(merged_df["Long-term RS"], merged_df["Short-term RS"], color="blue", marker="x")

    # Highlight the specific ticker with a star
    star = merged_df[merged_df["Ticker"] == "9618.HK"]
    if not star.empty:
        plt.scatter(star["Long-term RS"], star["Short-term RS"], color="gold", edgecolor="black", marker="*", s=100, label="9618.HK")

    # Set the axes labels
    plt.xlabel("Long-term RS")
    plt.ylabel("Short-term RS")

    # Set the title
    plt.title(f"Long-term vs short-term RS ({start_date} to {end_date})")

    # Set the legend
    plt.legend()

    # Show the plot
    plt.show()

In [ ]:
# Choose the stocks
stocks = ["CORZ", "VRT"]
for stock in stocks:
    df = get_df(stock, current_date)
    plot_close(stock, df)
    plot_volatility(stock, df, 252)
    plot_MFI_RSI(stock, df, 252)
    plot_stocks(["^GSPC", stock], current_date)

In [ ]:
# Get the stop loss and target price of a stock
stock = "CORZ"
df = get_df(stock, current_date)
current_close = df["Close"].iloc[-1]
stoploss, stoploss_pct, target, target_pct = stoploss_target(stock, 12.5, "2024-10-07")
print(f"Plan for {stock}.")
print(f"Current close: {round(current_close, 2)}.")
print(f"SL {stoploss} ({stoploss_pct}%) TP {target} ({target_pct}%)")

In [ ]:
# Calculate the new stop loss
price_avg = 12.9
stoploss_2r = price_avg * (1 - stoploss_pct / 100)
stoploss_2r = round(stoploss_2r, 2)
stoploss_1r = price_avg * (1 - stoploss_pct / 100 / 2)
stoploss_1r = round(stoploss_1r, 2)
stoploss_0r = price_avg
stoploss_0r = round(stoploss_0r, 2)
print(f"SL {stoploss_2r} ({stoploss_pct}%)")
print(f"SL {stoploss_1r} ({stoploss_pct / 2}%)")
print(f"SL {stoploss_0r} (0.0%)")

In [ ]:
show = 252 * 3
stocks = ["GC=F", "SI=F", "HG=F"]
metal_df = merge_stocks(stocks, current_date)
metal_df["Gold/Silver Ratio"] = metal_df["Close (GC=F)"] / metal_df["Close (SI=F)"]
metal_df["Gold/Copper Ratio"] = metal_df["Close (GC=F)"] / metal_df["Close (HG=F)"]
metal_df = calculate_ZScore(metal_df, ["Gold/Silver Ratio", "Gold/Copper Ratio"], 252)

# Restrict the dataframe
metal_df = metal_df[- show:]

# Create a figure with three subplots, one for the metal prices, one for the ratios, one for the ratios z-score
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 8), gridspec_kw={"height_ratios": [3, 1, 1]}, sharex=True)

# Plot the metal prices on the first subplot
close_goldfirst = metal_df["Close (GC=F)"].iloc[0]
close_silverfirst = metal_df["Close (SI=F)"].iloc[0]
close_copperfirst = metal_df["Close (HG=F)"].iloc[0]
ax1.plot(100 / close_goldfirst * metal_df["Close (GC=F)"], label="Gold (scaled)", color="gold")
ax1.plot(100 / close_silverfirst * metal_df["Close (SI=F)"], label="Silver (scaled)", color="silver")
ax1.plot(100 / close_copperfirst * metal_df["Close (HG=F)"], label="Copper (scaled)", color="peru")

# Set the label of the first subplot
ax1.set_ylabel("Price")

# Set the x limit of the first subplot
ax1.set_xlim(metal_df.index[0], metal_df.index[-1])

# Plot the ratios on the second subplot
goldsilver_ratio_first = metal_df["Gold/Silver Ratio"].iloc[0]
goldcopper_ratio_first = metal_df["Gold/Copper Ratio"].iloc[0]
ax2.plot(100 / goldsilver_ratio_first * metal_df["Gold/Silver Ratio"], color="silver")
ax2.plot(100 / goldcopper_ratio_first * metal_df["Gold/Copper Ratio"], color="peru")

# Set the y label of the second subplot
ax2.set_ylabel("Ratio wrt Gold")

# Plot the ratios z-score on the third subplot
ax3.plot(metal_df["Gold/Silver Ratio Z-Score"], color="silver")
ax3.plot(metal_df["Gold/Copper Ratio Z-Score"], color="peru")
ax3.axhline(y=2, linestyle="dotted", label="Undervalued", color="green")
ax3.axhline(y=-2, linestyle="dotted", label="Overvalued", color="red")

# Set the y label of the third subplot
ax3.set_ylabel("Ratio z-score")

# Set the x label
plt.xlabel("Date")

# Set the title
plt.suptitle(f"Metal prices comparison")

# Combine the legends and place them at the top subplot
handles, labels = ax1.get_legend_handles_labels()
handles += ax3.get_legend_handles_labels()[0]
labels += ax3.get_legend_handles_labels()[1]
ax1.legend(handles, labels)

# Adjust the spacing between subplots
plt.tight_layout()

# Save the plot
plt.savefig("Result/Figure/metalcompare.png", dpi=300)    

# Show the plot
plt.show()

In [ ]:
# Choose the stock
stock = "^HSI"

# Get the price data of the stock
df = get_df(stock, current_date)

# Add indicators
df = add_indicator(df)
df = calculate_ZScore(df, ["MFI", "RSI"], period=252*15)

# Save the data of the index to a .csv file
filename = f"Price data/{stock}_{current_date}.csv"
df.to_csv(filename)

periods = [5, 10, 15, 20, 30, 60]
for period in periods:
    df[f"Close {period} Later"] = df["Close"].shift(- period)
    df[f"{period} Days Return (%)"] = ((df[f"Close {period} Later"] / df["Close"]) - 1) * 100

# Filter for MFI/RSI Z-Score >= 2.5
df_MFIRSI_filter = df[(df["MFI Z-Score"] >= 2.5)]
print(df_MFIRSI_filter)

# Plot histogram
for period in periods:
    # Create a figure
    plt.figure(figsize=(10, 6))

    # Plot the histogram
    plt.hist(df_MFIRSI_filter[f"{period} Days Return (%)"].dropna(), bins=30, label=f'{period} Days Return (%)')

    # Calculate the mean
    mean = df_MFIRSI_filter.loc[:, f"{period} Days Return (%)"].mean()

    # Draw a vertical line at the mean
    plt.axvline(mean, color="red", linestyle="dashed", linewidth=1.5, label=f"Mean: {mean:.2f}%")

    # Set the y-axis ticks to integers
    y_ticks = np.arange(0, plt.ylim()[1] + 1, 1)
    plt.yticks(y_ticks)

    # Set the labels
    plt.xlabel("Return (%)")
    plt.ylabel("Count")

    # Set the title
    plt.title(rf"{period} days return when MFI Z-Score$\geq 2.5$ (%)")

    # Set the legend
    plt.legend()

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot
    plt.savefig(f"Result/Figure/{period}returnMFIZgeq2.5.png", dpi=300)

    # Show the plot
    plt.show()